In [ ]:
#Ex 1

from collections import deque

def aspirador_de_po():
    estado_inicial = ('sala1', 1, 1, 1)  # Sala atual, situaçãoSala1, situaçãoSala2, situaçãoSala3
    objetivo = ('sala1', 0, 0, 0)
    fronteira = deque([estado_inicial])
    explorados = set()

    def oper(acao, estado):
        # Define as operações para mudar o estado baseado na ação
        if acao == 'aspirar':
            if estado[0] == 'sala1' and estado[1] == 1:
                return ('sala1', 0, estado[2], estado[3])
            elif estado[0] == 'sala2' and estado[2] == 1:
                return ('sala2', estado[1], 0, estado[3])
            elif estado[0] == 'sala3' and estado[3] == 1:
                return ('sala3', estado[1], estado[2], 0)
        elif acao == 'entrarSala1':
            if estado[0] == 'sala2':
                return ('sala1', estado[1], estado[2], estado[3])
            elif estado[0] == 'sala3':
                return ('sala1', estado[1], estado[2], estado[3])
        elif acao == 'entrarSala2':
            if estado[0] == 'sala1':
                return ('sala2', estado[1], estado[2], estado[3])
            elif estado[0] == 'sala3':
                return ('sala2', estado[1], estado[2], estado[3])
        elif acao == 'entrarSala3':
            if estado[0] == 'sala1':
                return ('sala3', estado[1], estado[2], estado[3])
            elif estado[0] == 'sala2':
                return ('sala3', estado[1], estado[2], estado[3])
        return estado

    while fronteira:
        estado = fronteira.popleft()  # FIFO para evitar ciclos
        if estado == objetivo:
            return estado  # Solução encontrada

        if estado in explorados:
            continue

        explorados.add(estado)

        # Gerar novos estados
        acoes = ['aspirar', 'entrarSala1', 'entrarSala2', 'entrarSala3']
        for acao in acoes:
            novo_estado = oper(acao, estado)
            if novo_estado not in explorados and novo_estado not in fronteira:
                fronteira.append(novo_estado)

    return None  # Nenhuma solução encontrada

# Executar a busca
solucao = aspirador_de_po()
if solucao:
    print("Solução encontrada:", solucao)
else:
    print("Nenhuma solução encontrada.")

Solução encontrada: ('sala1', 0, 0, 0)


In [ ]:
#Ex 2
def is_goal(state):
  """Verifica se o estado atual é o objetivo (jarro de 4 litros com 2 litros)."""
  return state[1] == 2

def get_neighbors(state):
  """Gera os estados vizinhos a partir do estado atual."""
  j3, j4 = state
  neighbors = []

  # Encher o jarro de 3 litros
  neighbors.append((3, j4))
  # Encher o jarro de 4 litros
  neighbors.append((j3, 4))
  # Esvaziar o jarro de 3 litros
  neighbors.append((0, j4))
  # Esvaziar o jarro de 4 litros
  neighbors.append((j3, 0))

  # Transferir água do jarro de 3 para o de 4
  transfer = min(j3, 4 - j4)
  neighbors.append((j3 - transfer, j4 + transfer))
  # Transferir água do jarro de 4 para o de 3
  transfer = min(j4, 3 - j3)
  neighbors.append((j3 + transfer, j4 - transfer))

  return neighbors

def busca_jarros():
  """Realiza a busca em largura para encontrar a solução."""
  initial_state = (0, 0)
  visited = set()
  queue = [(initial_state, [])]  # (estado, caminho)

  while queue:
    state, path = queue.pop(0)

    if state in visited:
      continue

    visited.add(state)

    if is_goal(state):
      return path + [state]

    for neighbor in get_neighbors(state):
      if neighbor not in visited:
        queue.append((neighbor, path + [state]))

  return None  # Sem solução encontrada

solution = busca_jarros()

if solution:
  print("Solução encontrada:")
  for step in solution:
    print(step)
else:
  print("Não foi encontrada uma solução.")

Solução encontrada:
(0, 0)
(3, 0)
(0, 3)
(3, 3)
(2, 4)
(2, 0)
(0, 2)


In [ ]:
#exercicio 3
from collections import deque

def aplicar_acao(acao, estado):
    F, R, G, M = estado
    if acao == 'vaiSozinho':
        if F == 'e':
            return ('d', R, G, M)
        else:
            return ('e', R, G, M)
    elif acao == 'levaRaposa':
        if F == 'e' and R == 'e' and G != M:
            return ('d', 'd', G, M)
    elif acao == 'levaGalinha':
        if F == 'e' and G == 'e':
            return ('d', R, 'd', M)
    elif acao == 'levaMilho':
        if F == 'e' and M == 'e' and R != G:
            return ('d', R, G, 'd')
    elif acao == 'voltaSozinho':
        if F == 'd':
            return ('e', R, G, M)
        else:
            return ('d', R, G, M)
    elif acao == 'trazRaposa':
        if F == 'd' and R == 'd' and G != M:
            return ('e', 'e', G, M)
    elif acao == 'trazGalinha':
        if F == 'd' and G == 'd':
            return ('e', R, 'e', M)
    elif acao == 'trazMilho':
        if F == 'd' and M == 'd' and R != G:
            return ('e', R, G, 'e')
    return estado

def expandir(estado):
    acoes = [
        'vaiSozinho', 'levaRaposa', 'levaGalinha', 'levaMilho',
        'voltaSozinho', 'trazRaposa', 'trazGalinha', 'trazMilho'
    ]
    estados_futuros = []
    for acao in acoes:
        novo_estado = aplicar_acao(acao, estado)
        if novo_estado != estado and eh_estado_valido(novo_estado):
            estados_futuros.append(novo_estado)
    return estados_futuros

def eh_estado_valido(estado):
    F, R, G, M = estado
    if G == R and G != F:
        return False
    if G == M and G != F:
        return False
    return True

def busca(estado_inicial):
    objetivo = ('d', 'd', 'd', 'd')
    fila = deque([estado_inicial])
    visitados = set()
    caminho = {estado_inicial: None}

    while fila:
        estado_atual = fila.popleft()

        if estado_atual == objetivo:
            caminho_retorno = []
            while estado_atual is not None:
                caminho_retorno.append(estado_atual)
                estado_atual = caminho[caminho_retorno[-1]]
            return caminho_retorno[::-1]

        if estado_atual in visitados:
            continue

        visitados.add(estado_atual)

        for proximo_estado in expandir(estado_atual):
            if proximo_estado not in visitados:
                fila.append(proximo_estado)
                caminho[proximo_estado] = estado_atual

    return None

estado_inicial = ('e', 'e', 'e', 'e')
solucao = busca(estado_inicial)

print("Caminho para a solução:")
for passo in solucao:
    print(passo)


Caminho para a solução:
('e', 'e', 'e', 'e')
('d', 'e', 'd', 'e')
('e', 'e', 'd', 'e')
('d', 'e', 'd', 'd')
('e', 'e', 'e', 'd')
('d', 'd', 'e', 'd')
('e', 'd', 'e', 'd')
('d', 'd', 'd', 'd')


In [ ]:
#Ex 4

from collections import deque

def bfs_resolve_quebra_cabeca(estado_inicial, estado_objetivo):
    # Função para encontrar a posição do zero (espaço vazio)
    def encontra_posicao_vazia(estado):
        for i in range(3):
            for j in range(3):
                if estado[i][j] == 0:
                    return (i, j)

    # Função para gerar os estados filhos a partir de um estado atual
    def gera_estados_filhos(estado):
        i, j = encontra_posicao_vazia(estado)
        movimentos = []
        # Movimentos possíveis: cima, baixo, esquerda, direita
        direcoes = [(-1, 0), (1, 0), (0, -1), (0, 1)]
        for direcao in direcoes:
            novo_i, novo_j = i + direcao[0], j + direcao[1]
            if 0 <= novo_i < 3 and 0 <= novo_j < 3:
                # Cria um novo estado trocando a posição do zero com a peça adjacente
                novo_estado = [linha[:] for linha in estado]
                novo_estado[i][j], novo_estado[novo_i][novo_j] = novo_estado[novo_i][novo_j], novo_estado[i][j]
                movimentos.append(novo_estado)
        return movimentos

    # Inicializa a fila e o conjunto de visitados
    fila = deque([(estado_inicial, [])])  # Cada item da fila é uma tupla (estado, caminho até aqui)
    visitados = set()

    while fila:
        estado_atual, caminho = fila.popleft()

        # Se encontrou o estado objetivo, retorna o caminho
        if estado_atual == estado_objetivo:
            return caminho

        # Adiciona o estado atual ao conjunto de visitados
        estado_atual_tuple = tuple(tuple(linha) for linha in estado_atual)
        if estado_atual_tuple in visitados:
            continue
        visitados.add(estado_atual_tuple)

        # Gera estados filhos e adiciona à fila
        for filho in gera_estados_filhos(estado_atual):
            fila.append((filho, caminho + [filho]))

    return None  # Retorna None se não encontrar solução

# Exemplo de uso
estado_inicial = [
    [1, 2, 3],
    [4, 0, 5],
    [6, 7, 8]
]

estado_objetivo = [
    [1, 2, 3],
    [4, 5, 6],
    [7, 8, 0]
]

solucao = bfs_resolve_quebra_cabeca(estado_inicial, estado_objetivo)
if solucao:
    print("Solução encontrada em", len(solucao), "movimentos:")
    for passo in solucao:
        for linha in passo:
            print(linha)
        print()
else:
    print("Nenhuma solução encontrada.")


Solução encontrada em 14 movimentos:
[1, 2, 3]
[4, 5, 0]
[6, 7, 8]

[1, 2, 3]
[4, 5, 8]
[6, 7, 0]

[1, 2, 3]
[4, 5, 8]
[6, 0, 7]

[1, 2, 3]
[4, 5, 8]
[0, 6, 7]

[1, 2, 3]
[0, 5, 8]
[4, 6, 7]

[1, 2, 3]
[5, 0, 8]
[4, 6, 7]

[1, 2, 3]
[5, 6, 8]
[4, 0, 7]

[1, 2, 3]
[5, 6, 8]
[4, 7, 0]

[1, 2, 3]
[5, 6, 0]
[4, 7, 8]

[1, 2, 3]
[5, 0, 6]
[4, 7, 8]

[1, 2, 3]
[0, 5, 6]
[4, 7, 8]

[1, 2, 3]
[4, 5, 6]
[0, 7, 8]

[1, 2, 3]
[4, 5, 6]
[7, 0, 8]

[1, 2, 3]
[4, 5, 6]
[7, 8, 0]



In [ ]:
#Ex 5
import heapq

mapa = {
    'Arad': {'Sibiu': 140, 'Timisoara': 118, 'Zerind': 75},
    'Sibiu': {'Arad': 140, 'Fagaras': 99, 'Rimnicu Vilcea': 80},
    'Timisoara': {'Arad': 118, 'Lugoj': 111},
    'Zerind': {'Arad': 75, 'Oradea': 71},
    'Oradea': {'Zerind': 71, 'Sibiu': 151},
    'Fagaras': {'Sibiu': 99, 'Bucharest': 211},
    'Rimnicu Vilcea': {'Sibiu': 80, 'Craiova': 146, 'Pitesti': 97},
    'Lugoj': {'Timisoara': 111, 'Mehadia': 70},
    'Mehadia': {'Lugoj': 70, 'Drobeta': 75},
    'Drobeta': {'Mehadia': 75, 'Craiova': 120},
    'Craiova': {'Drobeta': 120, 'Rimnicu Vilcea': 146, 'Pitesti': 138},
    'Pitesti': {'Craiova': 138, 'Rimnicu Vilcea': 97, 'Bucharest': 101},
    'Bucharest': {'Pitesti': 101, 'Fagaras': 211}
}

heuristicas = {
    'Arad': 366,
    'Sibiu': 253,
    'Timisoara': 329,
    'Zerind': 374,
    'Oradea': 380,
    'Fagaras': 178,
    'Rimnicu Vilcea': 193,
    'Lugoj': 244,
    'Mehadia': 241,
    'Drobeta': 242,
    'Craiova': 160,
    'Pitesti': 98,
    'Bucharest': 0
}

def busca_gulosa(inicio, objetivo):
    fila = []
    heapq.heappush(fila, (heuristicas[inicio], inicio))
    visitados = set()
    caminho = {inicio: None}

    while fila:
        _, estado_atual = heapq.heappop(fila)

        if estado_atual == objetivo:
            caminho_retorno = []
            while estado_atual is not None:
                caminho_retorno.append(estado_atual)
                estado_atual = caminho[caminho_retorno[-1]]
            return caminho_retorno[::-1]

        if estado_atual in visitados:
            continue

        visitados.add(estado_atual)

        for vizinho, custo in mapa.get(estado_atual, {}).items():
            if vizinho not in visitados:
                prioridade = heuristicas[vizinho]
                heapq.heappush(fila, (prioridade, vizinho))
                if vizinho not in caminho:
                    caminho[vizinho] = estado_atual

    return None

inicio = 'Arad'
objetivo = 'Bucharest'
solucao = busca_gulosa(inicio, objetivo)

print("Caminho para a solução:")
if solucao:
    print(" -> ".join(solucao))
else:
    print("Nenhum caminho encontrado.")


Caminho para a solução:
Arad -> Sibiu -> Fagaras -> Bucharest


In [ ]:
# ex 6
import heapq

mapa = {
    'Arad': {'Sibiu': 140, 'Timisoara': 118, 'Zerind': 75},
    'Sibiu': {'Arad': 140, 'Fagaras': 99, 'Rimnicu Vilcea': 80},
    'Timisoara': {'Arad': 118, 'Lugoj': 111},
    'Zerind': {'Arad': 75, 'Oradea': 71},
    'Oradea': {'Zerind': 71, 'Sibiu': 151},
    'Fagaras': {'Sibiu': 99, 'Bucharest': 211},
    'Rimnicu Vilcea': {'Sibiu': 80, 'Craiova': 146, 'Pitesti': 97},
    'Lugoj': {'Timisoara': 111, 'Mehadia': 70},
    'Mehadia': {'Lugoj': 70, 'Drobeta': 75},
    'Drobeta': {'Mehadia': 75, 'Craiova': 120},
    'Craiova': {'Drobeta': 120, 'Rimnicu Vilcea': 146, 'Pitesti': 138},
    'Pitesti': {'Craiova': 138, 'Rimnicu Vilcea': 97, 'Bucharest': 101},
    'Bucharest': {'Pitesti': 101, 'Fagaras': 211}
}

heuristicas = {
    'Arad': 366,
    'Sibiu': 253,
    'Timisoara': 329,
    'Zerind': 374,
    'Oradea': 380,
    'Fagaras': 178,
    'Rimnicu Vilcea': 193,
    'Lugoj': 244,
    'Mehadia': 241,
    'Drobeta': 242,
    'Craiova': 160,
    'Pitesti': 98,
    'Bucharest': 0
}

def busca_a_star(inicio, objetivo):
    fila = []
    heapq.heappush(fila, (heuristicas[inicio], 0, inicio))
    visitados = set()
    caminho = {inicio: None}
    custo_acumulado = {inicio: 0}

    while fila:
        _, custo_atual, estado_atual = heapq.heappop(fila)

        if estado_atual == objetivo:
            caminho_retorno = []
            while estado_atual is not None:
                caminho_retorno.append(estado_atual)
                estado_atual = caminho[caminho_retorno[-1]]
            return caminho_retorno[::-1]

        if estado_atual in visitados:
            continue

        visitados.add(estado_atual)

        for vizinho, custo in mapa.get(estado_atual, {}).items():
            novo_custo = custo_acumulado[estado_atual] + custo
            if vizinho not in custo_acumulado or novo_custo < custo_acumulado[vizinho]:
                custo_acumulado[vizinho] = novo_custo
                prioridade = novo_custo + heuristicas[vizinho]
                heapq.heappush(fila, (prioridade, novo_custo, vizinho))
                if vizinho not in caminho:
                    caminho[vizinho] = estado_atual

    return None

inicio = 'Arad'
objetivo = 'Bucharest'
solucao = busca_a_star(inicio, objetivo)

print("Caminho para a solução:")
if solucao:
    print(" -> ".join(solucao))
else:
    print("Nenhum caminho encontrado.")


Caminho para a solução:
Arad -> Sibiu -> Rimnicu Vilcea -> Pitesti -> Bucharest


In [ ]:
#Ex 7
import heapq
import math

MOVES = [(-1, 0), (1, 0), (0, -1), (0, 1), (-1, -1), (-1, 1), (1, -1), (1, 1)]
COSTS = [10, 10, 10, 10, 14, 14, 14, 14]

def heuristic(a, b):
    return math.sqrt((a[0] - b[0]) ** 2 + (a[1] - b[1]) ** 2)

def astar(grid, start, goal):
    rows, cols = len(grid), len(grid[0])
    open_list = []
    heapq.heappush(open_list, (0 + heuristic(start, goal), 0, start))

    came_from = {}
    g_costs = {start: 0}

    while open_list:
        _, current_g, current = heapq.heappop(open_list)

        if current == goal:
            path = []
            while current in came_from:
                path.append(current)
                current = came_from[current]
            path.append(start)
            return path[::-1]

        for i, move in enumerate(MOVES):
            neighbor = (current[0] + move[0], current[1] + move[1])
            if 0 <= neighbor[0] < rows and 0 <= neighbor[1] < cols:
                if grid[neighbor[0]][neighbor[1]] == 1:
                    continue

                tentative_g = current_g + COSTS[i]

                if neighbor not in g_costs or tentative_g < g_costs[neighbor]:
                    came_from[neighbor] = current
                    g_costs[neighbor] = tentative_g
                    f = tentative_g + heuristic(neighbor, goal)
                    heapq.heappush(open_list, (f, tentative_g, neighbor))

    return None

grid = [
    [0, 0, 0, 0, 0],
    [0, 1, 1, 1, 0],
    [0, 0, 0, 1, 0],
    [0, 1, 0, 0, 0],
    [0, 0, 0, 0, 0]
]
start = (0, 0)
goal = (4, 4)

path = astar(grid, start, goal)
print("Caminho encontrado:", path)

Caminho encontrado: [(0, 0), (1, 0), (2, 1), (2, 2), (3, 3), (4, 4)]
